In [1]:
import os
import cv2
import shutil
import numpy as np
from tqdm import tqdm
from natsort import natsorted  # Ensure proper numerical ordering

# Define input and output directories
frames_base_dir = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/batman-frames"
keyshots_base_dir = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna"

# Ensure output directory exists
os.makedirs(keyshots_base_dir, exist_ok=True)

def compute_dynamic_threshold(hist_diffs):
    """
    Computes a dynamic threshold based on mean + standard deviation.
    Adjusts for scene complexity to improve keyshot selection.
    """
    if not hist_diffs:
        return 0.3  # Default threshold if no data available

    mean_diff = np.mean(hist_diffs)
    std_diff = np.std(hist_diffs)
    return mean_diff + (1.5 * std_diff)  # Adaptive thresholding

def extract_keyshots(sequence_dir, output_dir):
    """
    Extracts keyshots based on dynamic histogram differences.
    Adjusts the threshold based on scene changes.
    """
    frames = natsorted([f for f in os.listdir(sequence_dir) if f.endswith(".jpg")])
    keyshots = []

    if not frames:
        return keyshots

    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    prev_hist = None
    hist_diffs = []  # Store histogram differences

    # Compute histogram differences for all frames
    for frame in frames:
        frame_path = os.path.join(sequence_dir, frame)
        image = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)

        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()

        if prev_hist is not None:
            hist_diff = cv2.compareHist(prev_hist, hist, cv2.HISTCMP_BHATTACHARYYA)
            hist_diffs.append(hist_diff)

        prev_hist = hist

    # Compute dynamic threshold
    dynamic_threshold = compute_dynamic_threshold(hist_diffs)
    print(f"📌 Dynamic Threshold for {os.path.basename(sequence_dir)}: {dynamic_threshold:.3f}")

    # Extract keyshots based on dynamic threshold
    prev_hist = None
    for frame in tqdm(frames, desc=f"📸 Extracting Keyshots ({os.path.basename(sequence_dir)})", leave=False):
        frame_path = os.path.join(sequence_dir, frame)
        image = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)

        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()

        if prev_hist is None or cv2.compareHist(prev_hist, hist, cv2.HISTCMP_BHATTACHARYYA) > dynamic_threshold:
            keyshots.append(frame_path)
            prev_hist = hist

    # Save keyshots
    for idx, shot in enumerate(keyshots, 1):
        shutil.copy(shot, os.path.join(output_dir, f"keyshot_{idx}.jpg"))

    print(f"✅ Saved {len(keyshots)} keyshots in {output_dir}")

# Process each clip folder
for i, clip_folder in enumerate(natsorted(os.listdir(frames_base_dir))):
    clip_path = os.path.join(frames_base_dir, clip_folder)
    if os.path.isdir(clip_path):  # Ensure it's a directory
        output_clip_dir = os.path.join(keyshots_base_dir, f"clip-{i}")  # Save under clip-X format
        extract_keyshots(clip_path, output_clip_dir)

print("\n🎬 Dynamic Keyshot Extraction Completed for All Clips! 🚀")


📌 Dynamic Threshold for Batman (1989)-002_part_0: 0.071


✅ Saved 171 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-0
📌 Dynamic Threshold for Batman (1989)-002_part_1: 0.058


✅ Saved 36 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1
📌 Dynamic Threshold for Batman (1989)-002_part_2: 0.047


✅ Saved 1082 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-2
📌 Dynamic Threshold for Batman (1989)-002_part_3: 0.044


✅ Saved 219 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-3
📌 Dynamic Threshold for Batman (1989)-002_part_4: 0.022


...

✅ Saved 2 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1068
📌 Dynamic Threshold for Batman (1989)-002_part_1072: 0.040


✅ Saved 15 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1069
📌 Dynamic Threshold for Batman (1989)-002_part_1073: 0.039


✅ Saved 2 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1070
📌 Dynamic Threshold for Batman (1989)-002_part_1074: 0.037


✅ Saved 16 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1071
📌 Dynamic Threshold for Batman (1989)-002_part_1075: 0.032


✅ Saved 15 keyshots in /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/VLM-testing/dyna/clip-1072

🎬 Dynamic Keyshot Extraction Completed for All Clips! 🚀
